# Beispiel für Agent zur Buchung von Hotel und Flug

Diese Lösung hilft Ihnen dabei, Flugtickets und Hotels zu buchen. Das Szenario ist eine Reise vom London Heathrow LHR am 20. Februar 2024 nach New York JFK mit Rückflug am 27. Februar 2025, Economy-Class, ausschließlich mit British Airways. Ich möchte einen Aufenthalt in einem Hilton-Hotel in New York, bitte geben Sie die Kosten für den Flug und das Hotel an.


# Initialisieren des Azure AI Agent Service und Abrufen von Konfigurationsinformationen aus **.env**

### **.env**

Erstellen Sie eine .env-Datei

**.env** enthält die Verbindungszeichenfolge des Azure AI Agent Service, das von AOAI verwendete Modell und den entsprechenden Google API Search Service API, ENDPOINT usw.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Ihr Azure AI Agent Service Model Deployment-Name"

[**HINWEIS**] Sie benötigen ein Modell mit 100.000 Rate Limit (Tokens pro Minute) und eine Rate Limit von 600 (Anfragen pro Minute)

  Sie können das Modell in Microsoft Foundry - Modell und Endpunkt erhalten.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Ihre Azure AI Agent Service Projekt-Verbindungszeichenfolge"

  Sie können die Projektverbindungszeichenfolge in Ihrer Projektübersicht im AI Foundry Portal-Bildschirm erhalten.

- **SERPAPI_SEARCH_API_KEY** = "Ihr SERPAPI Search API SCHLÜSSEL"
- **SERPAPI_SEARCH_ENDPOINT** = "Ihr SERPAPI Search Endpunkt"

Um den Model Deployment Name und die Projektverbindungszeichenfolge des Azure AI Agent Service zu erhalten, müssen Sie den Azure AI Agent Service erstellen. Es wird empfohlen, [diese Vorlage](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) zu verwenden, um ihn direkt zu erstellen (***Hinweis:*** Der Azure AI Agent Service ist derzeit nur in eingeschränkten Regionen verfügbar. Es wird empfohlen, dass Sie sich an [diesen Link](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) halten, um die Region einzustellen)

Der Agent benötigt Zugriff auf SERPAPI. Es wird empfohlen, sich über [diesen Link](https://serpapi.com/searches) zu registrieren. Nach der Registrierung können Sie einen einzigartigen API SCHLÜSSEL und ENDPOINT erhalten.


# Einrichtung 

Um dieses Notebook auszuführen, müssen Sie sicherstellen, dass Sie die erforderlichen Bibliotheken installiert haben, indem Sie `pip install -r requirements.txt` ausführen.


In [ ]:
from semantic_kernel import __version__

__version__

Ihre Semantic Kernel-Version sollte mindestens 1.27.2 sein.


Laden Sie Ihre .env-Dateieinstellungen und Ressourcen. Bitte stellen Sie sicher, dass Sie Ihre Schlüssel und Einstellungen hinzugefügt und eine lokale .env-Datei erstellt haben.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Bei Azure anmelden

Sie müssen sich nun bei Azure anmelden. Öffnen Sie ein Terminal und führen Sie den folgenden Befehl aus:

```bash
az login
```

Dieser Befehl fordert Sie auf, Ihre Azure-Anmeldeinformationen einzugeben, damit der Azure AI Agent Dienst korrekt funktioniert.


# Erklärung:
Dies ist eine Variable, die den API-Schlüssel für den Zugriff auf einen SERP (Search Engine Results Page) API-Dienst speichert. Ein API-Schlüssel ist ein eindeutiger Identifikator, der verwendet wird, um Anfragen, die mit Ihrem Konto verknüpft sind, zu authentifizieren.

Zweck: Der Zweck dieser Zeile ist es, den API-Schlüssel in einer Variable zu speichern, damit er zur Authentifizierung von Anfragen an den SERP API-Dienst verwendet werden kann. Der API-Schlüssel wird benötigt, um auf den Dienst zuzugreifen und Suchvorgänge durchzuführen.
So erhalten Sie einen SERP-API-Schlüssel: Um einen SERP-API-Schlüssel zu erhalten, folgen Sie diesen allgemeinen Schritten auf https://serpapi.com (die genauen Schritte können je nach dem spezifischen SERP-API-Dienst, den Sie verwenden, variieren):

Wählen Sie einen SERP-API-Dienst: Es gibt mehrere verfügbare SERP-API-Dienste, wie SerpAPI, Google Custom Search JSON API und andere. Wählen Sie den Dienst, der am besten zu Ihren Anforderungen passt.

Registrieren Sie sich für ein Konto: Gehen Sie auf die Website des gewählten SERP-API-Dienstes und melden Sie sich für ein Konto an. Möglicherweise müssen Sie einige grundlegende Informationen angeben und Ihre E-Mail-Adresse verifizieren.

Erstellen Sie einen API-Schlüssel: Nachdem Sie sich registriert haben, melden Sie sich bei Ihrem Konto an und navigieren Sie zum API-Bereich oder Dashboard. Suchen Sie nach einer Option, um einen neuen API-Schlüssel zu erstellen oder zu generieren.
Kopieren Sie den API-Schlüssel in Ihre .env-Datei.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Erklärung:
BASE_URL: Dies ist eine Variable, die die Basis-URL für den SERP-API-Endpunkt speichert. Der Variablenname BASE_URL ist eine Konvention, die anzeigt, dass diese URL der Ausgangspunkt für API-Anfragen ist.
'https://serpapi.com/search':

Dies ist der tatsächliche URL-String, der der BASE_URL-Variablen zugewiesen wird. Er stellt den Endpunkt dar, um Suchanfragen mit der SERP API durchzuführen.

# Zweck:
Der Zweck dieser Zeile ist es, eine Konstante zu definieren, die die Basis-URL für die SERP API enthält. Diese URL wird als Ausgangspunkt zum Erstellen von API-Anfragen verwendet, um Suchvorgänge durchzuführen.

# Verwendung:
Durch die Definition der Basis-URL in einer Variable können Sie sie einfach im gesamten Code wiederverwenden, wann immer Sie Anfragen an die SERP API stellen müssen. Dies macht Ihren Code wartbarer und reduziert das Risiko von Fehlern durch das mehrfache Hardcodieren der URL an verschiedenen Stellen. Das aktuelle Beispiel ist https://serpapi.com/search?engine=bing, das die Bing-Such-API verwendet. Verschiedene APIs können unter https://Serpapi.com ausgewählt werden.


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Erklärung:

Hier befindet sich Ihr Plugin-Code.

Klassendefinition: `class BookingPlugin`: Definiert eine Klasse namens BookingPlugin, die Methoden zum Buchen von Hotels und Flügen enthält.

Hotelbuchungsmethode:

- `@kernel_function(description="booking hotel")`: Ein Dekorator, der die Funktion als Kernel-Funktion zur Hotelbuchung beschreibt.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Definiert eine Methode zur Hotelbuchung mit annotierten Parametern und Rückgabetyp.

Die Methode erstellt ein Wörterbuch mit Parametern für die Hotelbuchungsanfrage und sendet eine GET-Anfrage an die SERP-API. Sie prüft den Antwortstatus und gibt die Hoteldetails zurück, wenn die Anfrage erfolgreich war, oder None, wenn die Anfrage fehlgeschlagen ist.

Flugbuchungsmethode:

- `@kernel_function(description="booking flight")`: Ein Dekorator, der die Funktion als Kernel-Funktion zur Flugbuchung beschreibt.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Definiert eine Methode zur Flugbuchung mit annotierten Parametern und Rückgabetyp.

Die Methode erstellt Wörterbücher mit Parametern für die Hin- und Rückflug-Anfragen und sendet GET-Anfragen an die SERP-API. Sie prüft den Antwortstatus und fügt die Fluginformationen zur Ergebniszeichenfolge hinzu, wenn die Anfrage erfolgreich war, oder gibt eine Fehlermeldung aus, wenn die Anfrage fehlgeschlagen ist. Die Methode gibt die Ergebniszeichenfolge mit den Fluginformationen zurück.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Erklärung:
Importanweisungen: Importieren Sie die notwendigen Module für Azure-Anmeldeinformationen, AI-Agenten, Chatnachrichteninhalt, Autorenrolle und Kernel-Funktionsdekoration.

Asynchroner Kontextmanager: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Dies richtet einen asynchronen Kontextmanager ein, um Azure-Anmeldeinformationen zu verwalten und einen AI-Agenten-Client zu erstellen.

Agentenname und Anweisungen: 
- `AGENT_NAME = "BookingAgent"`: Definiert den Namen des Agenten.
- `AGENT_INSTRUCTIONS = """..."""`: Gibt detaillierte Anweisungen für den Agenten, wie Buchungsanfragen gehandhabt werden sollen.

Agentendefinition erstellen: `agent_definition = await client.agents.create_agent(...)`: Erzeugt eine Agentendefinition mit dem angegebenen Modell, Namen und den Anweisungen.

AzureAI-Agent erstellen: `agent = AzureAIAgent(...)`: Erstellt einen AzureAI-Agenten mithilfe des Clients, der Agentendefinition und des definierten Plugins.

Thread erstellen: `thread: AzureAIAgentThread | None = None`: Erstellt einen Thread für den Agenten. Es ist nicht erforderlich, zuerst einen Thread zu erstellen – wenn der Wert `None` angegeben wird, wird bei der ersten Aufruf ein neuer Thread erstellt und als Teil der Antwort zurückgegeben.

Benutzereingaben: `user_inputs = ["..."]`: Definiert eine Liste von Benutzereingaben, die vom Agenten verarbeitet werden sollen.

Im finally-Block werden der Thread und der Agent gelöscht, um Ressourcen aufzuräumen.


# Authentifizierung

Die Klasse `DefaultAzureCredential` ist Teil des Azure SDK für Python. Sie bietet eine standardmäßige Möglichkeit, sich bei Azure-Diensten zu authentifizieren. Dabei versucht sie, sich mithilfe mehrerer Methoden in einer bestimmten Reihenfolge zu authentifizieren, wie z. B. Umgebungsvariablen, verwaltete Identität und Azure CLI-Anmeldeinformationen.

Asynchrone Operationen: Das aio-Modul zeigt an, dass die Klasse DefaultAzureCredential asynchrone Operationen unterstützt. Das bedeutet, dass Sie sie mit asyncio verwenden können, um nicht blockierende Authentifizierungsanfragen durchzuführen.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, sollten Sie beachten, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner Ausgangssprache gilt als maßgebliche Quelle. Für wichtige Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die aus der Nutzung dieser Übersetzung entstehen.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
